In [1]:
# Load libraries 
import yfinance as yf
import datetime as dt
import pandas as pd
from metalpriceapi.client import Client

In [ ]:
# Starting date and ending date of the period within which data is collected
START_DATE = dt.datetime(2014, 1, 1)
END_DATE = dt.datetime(2024, 2, 6+1)

Collect exchange rates USD/VND using Yahoo Finance API

In [ ]:
# Download exchange rates USD/VND
ticker_exchange_rate ="VND=X"
data_exchange_rates = yf.download(ticker_exchange_rate, START_DATE, END_DATE)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Extract the Adj Close column of exchange rate
data_exchange_rates = data_exchange_rates.copy()
data_exchange_rates = data_exchange_rates[['Adj Close']]
data_exchange_rates = data_exchange_rates.rename(columns={"Adj Close": "exchange_rate"})

In [ ]:
data_exchange_rates

,exchange_rate
Date,
2014-01-01,20835.0
2014-01-02,21055.0
2014-01-03,21050.0
2014-01-06,21050.0
2014-01-07,21045.0
...,...
2024-01-31,24395.0
2024-02-01,24415.0
2024-02-02,24400.0


Collect gold price XAUUSD using Metal price API

In [ ]:
api_key = 'API_KEY' # insert your API key 
client = Client(api_key)

In [ ]:
# The limit of each API call is 365 days so we have to split our periods into several parts 
date_partition = []

start_temp = START_DATE
end_temp = START_DATE

while (pd.to_datetime(END_DATE) - pd.to_datetime(end_temp)).days >= 365: # check if the interval is larger than or equal to 365 days
  end_temp = start_temp + pd.Timedelta(days=365)  # Define period which will be extracted, the interval is 365 days 
  date_partition.append([start_temp.strftime('%Y-%m-%d'), end_temp.strftime('%Y-%m-%d')])   # Collect data within the period 
  start_temp = end_temp + pd.Timedelta(days=1)   # Reset the start date for the subsequent period 
else:
  # Collect data for the remaining days (less than 365 days)
  start_temp = end_temp + pd.Timedelta(days=1)
  end_temp = END_DATE
  date_partition.append([start_temp.strftime('%Y-%m-%d'), end_temp.strftime('%Y-%m-%d')])


In [ ]:
# Start date and end date of each subsets 
date_partition

[['2014-01-01', '2015-01-01'],
 ['2015-01-02', '2016-01-02'],
 ['2016-01-03', '2017-01-02'],
 ['2017-01-03', '2018-01-03'],
 ['2018-01-04', '2019-01-04'],
 ['2019-01-05', '2020-01-05'],
 ['2020-01-06', '2021-01-05'],
 ['2021-01-06', '2022-01-06'],
 ['2022-01-07', '2023-01-07'],
 ['2023-01-08', '2024-01-08'],
 ['2024-01-09', '2024-02-07']]

In [ ]:
# Iteratively retrieve data within the periods of subsets 
lst_retrieved_xauusd = []
for time_pair in date_partition:
  retrieved = client.timeframe(start_date=time_pair[0], end_date=time_pair[1], base='XAU', currencies=['USD'])
  lst_retrieved_xauusd.append(retrieved)

In [ ]:
# Initialize empty lists to store data
dates = []
rates = []

# Iterate over each dictionary in the list of retrieved valuyes 
for data in lst_retrieved_xauusd:
    rates_data = data['rates']
    for date, rate in rates_data.items():
        dates.append(date)
        rates.append(rate['USD'])

# Create DataFrame to store data 
data_xauusd = pd.DataFrame({'date': dates, 'xauusd': rates})

In [ ]:
data_xauusd

,date,xauusd
0,2014-01-01,1209.006691
1,2014-01-02,1224.088499
2,2014-01-03,1237.028331
3,2014-01-04,1237.462953
4,2014-01-05,1237.600787
...,...,...
3685,2024-02-03,2039.833460
3686,2024-02-04,2040.174296
3687,2024-02-05,2024.960063
3688,2024-02-06,2036.074351


In [ ]:
# Export data
data_exchange_rates.to_csv(f"../data/raw_datasets/data_exchange_rates.csv")
data_xauusd.to_csv(f"../data/raw_datasets/data_xauusd.csv")